In [1]:
import pandas as pd, sys, numpy as np

In [62]:
# add main directory to path in order to import other parts of the shelter animal outcomes project
sys.path.extend(['/Users/pc3sq/Google Drive/Data Science Library/!notebooks/shelter animal outcomes kaggle/'])

from src.pipeline import age_to_days

In [3]:
clean = pd.read_csv('../data/shelter-clean.csv')

In [16]:
transform = clean.copy()

In [5]:
# # previous transformation pipeline (for reference only; no longer in use)
# def transform_df(df):
#     dframe = df.copy()

#     dframe['Name'] = dframe['Name'].apply(lambda name: "hasName" if name!="noName" else name)

#     dframe = dframe.drop('OutcomeSubtype', axis=1)

#     dframe['AgeuponOutcome'] = dframe['AgeuponOutcome'].apply(age_to_days)

#     # create date features
#     dframe['DateTime'] = pd.to_datetime(dframe['DateTime'])
#     dframe['dow'] = dframe['DateTime'].apply(lambda dt: dt.dayofweek)
#     # dframe['dom']= dframe['DateTime'].apply(lambda dt: dt.day) # day of month might be useful
#     dframe['month'] = dframe['DateTime'].apply(lambda dt: dt.month)
#     dframe['doy'] = dframe['DateTime'].apply(lambda dt: dt.dayofyear)

#     # convert day of week to weekday names
#     weekdays = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
#     dframe['dow'] = dframe['dow'].apply(lambda dow: weekdays[dow])

#     # convert nominal month to month names
#     months = {1: 'Jan', 2: 'Feb', 3: 'March', 4: 'April', 5: 'May', 6: "June", 7: "July",\
#            8: 'Aug', 9: 'Sept', 10: "Oct", 11: 'Nov', 12: 'Dec'}
#     dframe['month'] = dframe['month'].apply(lambda month: months[month])

#     # extract mixed vs. pure for Breed
#     dframe['Breed'] = dframe['Breed'].apply(lambda breed: "Mix" if "Mix" in breed or "/" in breed else "Pure")

#     # get dummies
#     categoricals = ['Name', 'AnimalType', 'SexuponOutcome', 'Breed', 'dow', 'month']
#     dframe = pd.get_dummies(dframe, columns=categoricals)

#     # return only select columns
#     return dframe.drop(['AnimalID', 'DateTime', 'Color'], axis=1)

# <center>Update Tranformation Pipeline: Correct Binary Variable Dummying</center>

In [17]:
fix_binary = transform.copy()

In [18]:
# convert Breed feature to Pure vs. Mix
fix_binary['pureBreed'] = fix_binary['Breed'].apply(lambda breed: 0 if "Mix" in breed or "/" in breed else 1)

In [19]:
# check all pureBreeds == 1 and mixed == 0
sum(fix_binary[fix_binary['pureBreed']==1]['Breed'].apply(lambda breed: 1 if "Mix" in breed or "/" in breed else 0))

0

In [20]:
# extract hasName from Name
fix_binary['hasName'] = fix_binary['Name'].apply(lambda name: 0 if name=="noName" else 1)

In [21]:
# check pets with names == 1 and those without names == 0
sum(fix_binary[fix_binary['hasName']==1]['Name'].apply(lambda name: 1 if name=="noName" else 0))

0

In [22]:
# extract Dog or not dog from AnimalType
fix_binary['Dog'] = fix_binary['AnimalType'].apply(lambda specie: 1 if specie=="Dog" else 0)

In [23]:
# check all Dog==1 has not cats and all Dog==0 has no dogs
print sum(fix_binary[fix_binary['Dog']==1]['AnimalType'] == 'Cat')
print sum(fix_binary[fix_binary['Dog']==0]['AnimalType'] == 'Dog')

0
0


In [24]:
# make sure we drop our converted binary variables
fix_binary = fix_binary.drop(['Name', 'Breed', 'AnimalType'], axis=1)

In [25]:
# check remaining columns
fix_binary.columns

Index([u'AnimalID', u'DateTime', u'OutcomeType', u'OutcomeSubtype',
       u'SexuponOutcome', u'AgeuponOutcome', u'Color', u'pureBreed',
       u'hasName', u'Dog'],
      dtype='object')

## Extract Patterned From Color

In [26]:
patterned = fix_binary.copy()

In [28]:
# helpers for extracting Patterned from cats and dogs
def patterned_dog(color):
    dog_colors_with_patterns = { 'Black Brindle',
                                 'Black Brindle/Black',
                                 'Black Brindle/Brown',
                                 'Black Brindle/Brown Brindle',
                                 'Black Brindle/White',
                                 'Black/Black Brindle',
                                 'Black/Blue Tick',
                                 'Black/Brown Brindle',
                                 'Black/Tricolor',
                                 'Black/Yellow Brindle',
                                 'Blue Merle/Tricolor',
                                 'Blue Tick',
                                 'Blue Tick/Black',
                                 'Blue Tick/Brown',
                                 'Blue Tick/Brown Brindle',
                                 'Blue Tick/Red',
                                 'Blue Tick/Red Tick',
                                 'Blue Tick/Tan',
                                 'Blue Tick/White',
                                 'Blue/Brown Brindle',
                                 'Blue/Yellow Brindle',
                                 'Brown Brindle',
                                 'Brown Brindle/Black',
                                 'Brown Brindle/Blue',
                                 'Brown Brindle/Blue Cream',
                                 'Brown Brindle/Blue Tick',
                                 'Brown Brindle/Brown Brindle',
                                 'Brown Brindle/Brown Merle',
                                 'Brown Brindle/Red Tick',
                                 'Brown Brindle/Tan',
                                 'Brown Brindle/White',
                                 'Brown/Black Brindle',
                                 'Brown/Red Tick',
                                 'Brown/Tricolor',
                                  'Chocolate/Brown Brindle',
                                 'Chocolate/Red Tick',
                                 'Chocolate/Tricolor',
                                 'Cream/Red Tick',
                                 'Fawn/Brown Brindle',
                                 'Fawn/Tricolor',
                                 'Liver Tick',
                                 'Liver Tick/White',
                                 'Red Tick',
                                 'Red Tick/Black',
                                 'Red Tick/Blue Tick',
                                 'Red Tick/Brown',
                                 'Red Tick/Brown Merle',
                                 'Red Tick/Red',
                                 'Red Tick/Tan',
                                 'Red Tick/White',
                                 'Red/Red Tick',
                                 'Red/Tricolor',
                                 'Tan/Brown Brindle',
                                 'Tan/Tricolor',
                                 'Tan/Yellow Brindle',
                                 'Tricolor',
                                 'Tricolor/Black',
                                 'Tricolor/Blue',
                                 'Tricolor/Blue Merle',
                                 'Tricolor/Brown',
                                 'Tricolor/Brown Merle',
                                 'Tricolor/Chocolate',
                                 'Tricolor/Silver',
                                 'Tricolor/Tan',
                                 'Tricolor/White',
                                 'White/Black Brindle',
                                 'White/Blue Tick',
                                  'White/Brown Brindle',
                                 'White/Red Tick',
                                 'White/Tricolor',
                                 'White/Yellow Brindle',
                                 'Yellow Brindle',
                                 'Yellow Brindle/Blue',
                                 'Yellow Brindle/Tan',
                                 'Yellow Brindle/White' } 

    if color in dog_colors_with_patterns:
        return 1
    else:
        return 0

def patterned_cat(color):
    cat_colors_with_patterns = {'Agouti',
                                'Agouti/Brown Tabby',
                                'Brown Tiger',
                                'Black Tiger/White',
                                'Orange Tiger',
                                'Tricolor',
                                'Calico/Tricolor',
                                'Tricolor/Calico',
                                'Black Smoke/White',
                                'Black Smoke',
                                'Blue Smoke',
                                'Black/Black Smoke',
                                'Black Smoke/Brown Tabby',
                                'Chocolate/Black Smoke',
                                'White/Black Smoke',
                                'Tortie/Black Smoke',
                                'Cream Tabby',
                                'Blue Tabby',
                                'Orange Tabby/White',
                                'Brown Tabby',
                                'Orange Tabby',
                                'Cream Tabby/White',
                                'Blue Tabby/White',
                                'Brown Tabby/White',
                                'Black Tabby/White',
                                'White/Cream Tabby',
                                'White/Blue Tabby',
                                'White/Brown Tabby',
                                'White/Orange Tabby',
                                'Gray Tabby',
                                'Blue Tabby/Orange',
                                'Black Tabby',
                                'Calico/Brown Tabby',
                                'Silver Tabby',
                                'White/Black Tabby',
                                'Black/Silver Tabby',
                                'Blue Tabby/Blue Cream',
                                'Brown Tabby/Black',
                                'Gray Tabby/White',
                                'Black Tabby/Orange',
                                'Silver Tabby/White',
                                'Black Smoke/Brown Tabby',
                                'Blue Tabby/Tan',
                                'Black/Black Tabby',
                                'Brown Tabby/Gray',
                                'Brown/Black Tabby',
                                'Lynx Point/Brown Tabby',
                                'Orange Tabby/Black',
                                'Brown Tabby/Brown',
                                'White/Gray Tabby',
                                'Calico/Orange Tabby',
                                'Agouti/Brown Tabby',
                                'Brown Tabby/Tortie',
                                'Lynx Point/Gray Tabby',
                                'Blue Tabby/Cream',
                                'Silver Tabby/Black',
                                'Orange Tabby/Orange',
                                'Brown Tabby/Orange',
                                'Blue Cream/Blue Tabby',
                                'Calico/Blue Tabby',
                                'Brown Tabby/Gray Tabby',
                                'Lynx Point/White',
                                'Lynx Point',
                                'Tortie Point/Lynx Point',
                                'White/Lynx Point',
                                'Lynx Point/Brown Tabby',
                                'Silver Lynx Point',
                                'Lynx Point/Gray Tabby',
                                'Flame Point',
                                'Blue Point',
                                'Seal Point',
                                'Lynx Point/White',
                                'Lilac Point',
                                'Tortie Point',
                                'Lynx Point',
                                'Calico Point',
                                'Tortie Point/Lynx Point',
                                'White/Chocolate Point',
                                'Calico Point/White',
                                'Seal Point/White',
                                'Chocolate Point/White',
                                'White/Lynx Point',
                                'Blue Point/White',
                                'Chocolate Point',
                                'Seal Point/Brown',
                                'Lynx Point/Brown Tabby',
                                'Silver Lynx Point',
                                'Tortie Point/White',
                                'White/Flame Point',
                                'Cream/Seal Point',
                                'Lynx Point/Gray Tabby',
                                'Calico',
                                'Tortie/Calico',
                                'Calico Point',
                                'Calico/White',
                                'Calico Point/White',
                                'Calico/Brown Tabby',
                                'Calico/Black',
                                'Calico/Tricolor',
                                'Calico/Brown',
                                'Torbie/Calico',
                                'Calico/Orange Tabby',
                                'White/Calico',
                                'Calico/Blue Cream',
                                'Calico/Blue Tabby',
                                'Tricolor/Calico',
                                'Tortie',
                                'Tortie/Calico',
                                'Tortie Point',
                                'Tortie/White',
                                'Tortie Point/Lynx Point',
                                'Tortie/Blue Cream',
                                'Blue/Tortie',
                                'Tortie/Orange',
                                'Tortie/Brown',
                                'Tortie/Black',
                                'Tortie Point/White',
                                'Brown Tabby/Tortie',
                                'Tortie/Black Smoke',
                                'Blue Cream/Tortie',
                                'Torbie',
                                'Torbie/White',
                                'Torbie/Brown',
                                'Torbie/Blue Cream',
                                'Torbie/Calico'}
    
    if color in cat_colors_with_patterns:
        return 1
    else:
        return 0

In [30]:
patterned

,AnimalID,DateTime,OutcomeType,OutcomeSubtype,SexuponOutcome,AgeuponOutcome,Color,pureBreed,hasName,Dog
0,A671945,2014-02-12 18:22:00,Return_to_owner,noSubOutcome,Neutered Male,1 year,Brown/White,0,1,1
1,A656520,2013-10-13 12:44:00,Euthanasia,Suffering,Spayed Female,1 year,Cream Tabby,0,1,0
2,A686464,2015-01-31 12:28:00,Adoption,Foster,Neutered Male,2 years,Blue/White,0,1,1
3,A683430,2014-07-11 19:09:00,Transfer,Partner,Intact Male,3 weeks,Blue Cream,0,0,0
4,A667013,2013-11-15 12:52:00,Transfer,Partner,Neutered Male,2 years,Tan,0,0,1
5,A677334,2014-04-25 13:04:00,Transfer,Partner,Intact Female,1 month,Black/Tan,0,1,1
6,A699218,2015-03-28 13:11:00,Transfer,Partner,Intact Male,3 weeks,Blue Tabby,0,1,0
7,A671784,2014-02-04 17:17:00,Adoption,noSubOutcome,Spayed Female,5 months,Red/White,0,1,1
8,A677747,2014-05-03 07:48:00,Adoption,Offsite,Spayed Female,1 year,White,1,0,1
9,A666320,2013-11-04 14:48:00,Adoption,noSubOutcome,Spayed Female,2 years,Silver,0,0,1


In [31]:
# extract Patterned for dogs and cats
patterned['Patterned'] = [patterned_dog(row['Color']) if row['Dog']==1 else patterned_cat(row['Color']) for i, row in patterned.iterrows()]

In [34]:
# quick view of Patterned values
patterned['Patterned'][:10]

0    0
1    1
2    0
3    0
4    0
5    0
6    1
7    0
8    0
9    0
Name: Patterned, dtype: int64

In [37]:
# percentage of dogs that are patterned
sum(patterned[patterned['Dog']==1]['Patterned']==1) / float(patterned[patterned['Dog']==1].shape[0])

0.13075435203094776

In [39]:
# percentage of cats that are patterned
sum(patterned[patterned['Dog']==0]['Patterned']==1) / float(patterned[patterned['Dog']==0].shape[0])

0.67333201346268068

In [36]:
# overall percentage of patterned animals in dataset
sum(patterned['Patterned']==1) / float(patterned.shape[0])

0.34476026862408249

## Extract Multi-Colored From Color

In [92]:
# copy of an instance of transformed dframe until now
multicolored = patterned.copy()

In [93]:
# helpers for extracting Patterned from cats and dogs
def multi_dog(color):
    solid_dog_colors = { 'Tan',
                         'White',
                         'Silver',
                         'Brown',
                         'Black',
                         'Yellow',
                         'Buff',
                         'Red',
                         'Brown Brindle',
                         'Gray',
                         'Blue Merle',
                         'Sable',
                         'Cream',
                         'Chocolate',
                         'Fawn',
                         'Blue',
                         'Apricot',
                         'Yellow Brindle',
                         'Gold',
                         'Blue Tick',
                         'Red Merle',
                         'Black Brindle',
                         'Brown Merle',
                         'Red Tick',
                         'Liver Tick',
                         'Blue Tiger',
                         'Orange',
                         'Brown Tiger',
                         'Black Smoke' }

    if color in solid_dog_colors:
        return 0
    else:
        return 1

def multi_cat(color):
    solid_cat_colors = { 'Apricot',
                         'Black',
                         'Black Brindle',
                         'Black Smoke',
                         'Blue',
                         'Blue Merle',
                         'Blue Tick',
                         'Blue Tiger',
                         'Brown',
                         'Brown Brindle',
                         'Brown Merle',
                         'Brown Tiger',
                         'Buff',
                         'Chocolate',
                         'Cream',
                         'Fawn',
                         'Gold',
                         'Gray',
                         'Liver Tick',
                         'Orange',
                         'Red',
                         'Red Merle',
                         'Red Tick',
                         'Sable',
                         'Silver',
                         'Tan',
                         'Tricolor',
                         'White',
                         'Yellow',
                         'Yellow Brindle' }

    if color in solid_cat_colors:
        return 0
    else:
        return 1

In [94]:
# extract multi-colored for dogs and cats
multicolored['Multi-Colored'] = [multi_dog(row['Color']) if row['Dog']==1 else multi_cat(row['Color']) for i, row in multicolored.iterrows()]

In [95]:
# return indices of where cats are solid colored
idx_solid_colored_cats = multicolored[multicolored['Dog']==0]['Color'].isin([ 'Apricot',
                                                                                 'Black',
                                                                                 'Black Brindle',
                                                                                 'Black Smoke',
                                                                                 'Blue',
                                                                                 'Blue Merle',
                                                                                 'Blue Tick',
                                                                                 'Blue Tiger',
                                                                                 'Brown',
                                                                                 'Brown Brindle',
                                                                                 'Brown Merle',
                                                                                 'Brown Tiger',
                                                                                 'Buff',
                                                                                 'Chocolate',
                                                                                 'Cream',
                                                                                 'Fawn',
                                                                                 'Gold',
                                                                                 'Gray',
                                                                                 'Liver Tick',
                                                                                 'Orange',
                                                                                 'Red',
                                                                                 'Red Merle',
                                                                                 'Red Tick',
                                                                                 'Sable',
                                                                                 'Silver',
                                                                                 'Tan',
                                                                                 'Tricolor',
                                                                                 'White',
                                                                                 'Yellow',
                                                                                 'Yellow Brindle' ])

In [96]:
# check no solid colored cats have Multi-Colored value of 1
print sum(multicolored[multicolored['Dog']==0][idx_solid_colored_cats]['Multi-Colored']==1)

#check no multi-colored cats have Multi-Colored value of 0
print sum(multicolored[multicolored['Dog']==0][idx_solid_colored_cats==False]['Multi-Colored']==0)

0
0


In [97]:
idx_solid_colored_dogs = multicolored[multi_colored['Dog']==1]['Color'].isin([ 'Tan',
                                                                                 'White',
                                                                                 'Silver',
                                                                                 'Brown',
                                                                                 'Black',
                                                                                 'Yellow',
                                                                                 'Buff',
                                                                                 'Red',
                                                                                 'Brown Brindle',
                                                                                 'Gray',
                                                                                 'Blue Merle',
                                                                                 'Sable',
                                                                                 'Cream',
                                                                                 'Chocolate',
                                                                                 'Fawn',
                                                                                 'Blue',
                                                                                 'Apricot',
                                                                                 'Yellow Brindle',
                                                                                 'Gold',
                                                                                 'Blue Tick',
                                                                                 'Red Merle',
                                                                                 'Black Brindle',
                                                                                 'Brown Merle',
                                                                                 'Red Tick',
                                                                                 'Liver Tick',
                                                                                 'Blue Tiger',
                                                                                 'Orange',
                                                                                 'Brown Tiger',
                                                                                 'Black Smoke' ])

In [98]:
# check no solid colored dogs have Multi-Colored value of 1
print sum(multicolored[multicolored['Dog']==1][idx_solid_colored_dogs]['Multi-Colored']==1)

# check no multi-colored dogs have a Multi-Colored value of 0
print sum(multicolored[multicolored['Dog']==1][idx_solid_colored_dogs == False]['Multi-Colored'] == 0)

0
0


## Apply Remaining Transformations

In [105]:
final = multicolored.copy()

In [106]:
# convert all values in AgeuponOutcome to days
final['AgeuponOutcome'] = final['AgeuponOutcome'].apply(age_to_days)

# get dummies for non-binary categorical variables
categoricals = ['SexuponOutcome']
final = pd.get_dummies(final, columns=categoricals)

final = final.drop(['AnimalID', 'DateTime', 'Color', 'OutcomeSubtype'], axis=1)

In [107]:
final.columns

Index([u'OutcomeType', u'AgeuponOutcome', u'pureBreed', u'hasName', u'Dog',
       u'Patterned', u'Multi-Colored', u'SexuponOutcome_Intact Female',
       u'SexuponOutcome_Intact Male', u'SexuponOutcome_Neutered Male',
       u'SexuponOutcome_Spayed Female'],
      dtype='object')

# <center>New Transformation Pipeline</center>

Comments have been included about the DateTime column. DateTime should be treated as a type of target leakage because like the OutcomeSubtype column that has already been removed, it contains information unknown to the model until at the time of the outcome. I have chosen to remove it from the model.

While DateTime shouldn't be included as a predictor variable due to target leakage, it is nevertheless important to study the outcome DateTime to see if there is a correlation between some time feature and an outcome. Some animals may be more vulnerable to unfavorable outcomes during certain months or days of the week. Or perhaps there is a seasonality to adoption rates, that there are more adoptions when the weather is nice each year. Features extracted from DateTime can certainly be explored further for insights.

In [101]:
# new transform pipeline

def transform_pipeline(df):
    dframe = df.copy()
    
    # extract Dog or not dog from AnimalType
    dframe['Dog'] = dframe['AnimalType'].apply(lambda specie: 1 if specie=="Dog" else 0)
    
    # extract mixed vs. pure for Breed
    dframe['pureBreed'] = dframe['Breed'].apply(lambda breed: 0 if "Mix" in breed or "/" in breed else 1)
    
    # extract hasName from Name
    dframe['hasName'] = dframe['Name'].apply(lambda name: 0 if name=="noName" else 1)

    # extract Multi-colored from Color for dogs and cats
    dframe['Multi-Colored'] = [multi_dog(row['Color']) if row['Dog']==1 else multi_cat(row['Color']) for i, row in dframe.iterrows()]
    
    # extract Patterned from Color for dogs and cats
    dframe['Patterned'] = [patterned_dog(row['Color']) if row['Dog']==1 else patterned_cat(row['Color']) for i, row in dframe.iterrows()]

    # convert all values in AgeuponOutcome to days
    dframe['AgeuponOutcome'] = dframe['AgeuponOutcome'].apply(age_to_days)

    # get dummies for non-binary categorical variables
    categoricals = ['SexuponOutcome']
    dframe = pd.get_dummies(dframe, columns=categoricals)

    # return only select columns
    # dropping OutcomeSubtype due to target leakage
    return dframe.drop(['AnimalID', 'AnimalType', 'Breed', 'Name', 'DateTime', 'Color', 'OutcomeSubtype'], axis=1)

In [102]:
test_new_pipeline = transform_pipeline(clean)

In [108]:
# check that dframe produced by transform_pipeline() == dframe manually produced in this notebook
test_new_pipeline.shape == final.shape

True

In [109]:
# check that dframe columns produced by transform_pipeline() == dframe columns manually produced in this notebook
sorted(test_new_pipeline.columns) == sorted(final.columns)

True

In [110]:
## saved new transformed dataset to data folder
# test_new_pipeline.to_csv('../data/transformed-shelter-2016-07-28.csv', index=False)